## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-24-16-56-14 +0000,nypost,A cat named Leonardo da Pinchy doesn’t want yo...,https://nypost.com/2025/07/24/world-news/a-cat...
1,2025-07-24-16-54-32 +0000,nyt,The Fed Renovations Driving Trump’s Anger at J...,https://www.nytimes.com/2025/07/24/business/tr...
2,2025-07-24-16-50-39 +0000,bbc,Thousands unable to make calls as EE and BT ne...,https://www.bbc.com/news/articles/cnvmvqrnq7go
3,2025-07-24-16-48-22 +0000,nyt,Josh Shapiro Says Zohran Mamdani Fails to Cond...,https://www.nytimes.com/2025/07/24/us/politics...
4,2025-07-24-16-46-06 +0000,nypost,AG Pam Bondi launches ‘strike force’ after dec...,https://nypost.com/2025/07/24/us-news/pam-bond...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2334/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
15,trump,58
246,trade,16
46,russia,15
386,new,15
87,epstein,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
229,2025-07-24-01-00-00 +0000,wsj,President Trump’s push to introduce a new stan...,https://www.wsj.com/economy/trade/trump-trade-...,135
293,2025-07-23-21-45-42 +0000,nypost,Third round of Ukraine-Russia peace talks end ...,https://nypost.com/2025/07/23/us-news/ukraine-...,126
367,2025-07-23-18-03-36 +0000,nyt,How Japan Clinched a Trade Deal With Trump Jus...,https://www.nytimes.com/2025/07/23/business/tr...,108
55,2025-07-24-15-36-00 +0000,wsj,President Trump will visit the Federal Reserve...,https://www.wsj.com/economy/central-banking/tr...,108
210,2025-07-24-03-03-24 +0000,nypost,Appeals court blocks Trump’s effort to end bir...,https://nypost.com/2025/07/23/us-news/appeals-...,107


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
229,135,2025-07-24-01-00-00 +0000,wsj,President Trump’s push to introduce a new stan...,https://www.wsj.com/economy/trade/trump-trade-...
293,68,2025-07-23-21-45-42 +0000,nypost,Third round of Ukraine-Russia peace talks end ...,https://nypost.com/2025/07/23/us-news/ukraine-...
22,63,2025-07-24-16-28-01 +0000,nypost,Bryan Kohberger sent to Idaho’s hellhole max s...,https://nypost.com/2025/07/24/us-news/bryan-ko...
47,54,2025-07-24-15-56-52 +0000,nyt,"MyPillow Founder, Mike Lindell, Will Not Pay o...",https://www.nytimes.com/2025/07/24/us/election...
186,54,2025-07-24-07-33-58 +0000,nypost,Thailand and Cambodia exchange fire in new bor...,https://nypost.com/2025/07/24/world-news/thail...
115,46,2025-07-24-13-18-09 +0000,nyt,House Panel Votes to Subpoena Justice Dept. fo...,https://www.nytimes.com/2025/07/23/us/politics...
94,42,2025-07-24-14-19-43 +0000,nypost,Tesla stock plunges as Elon Musk warns company...,https://nypost.com/2025/07/24/business/tesla-s...
126,39,2025-07-24-12-13-24 +0000,nypost,American Airlines flight attendant who admitte...,https://nypost.com/2025/07/24/us-news/american...
12,38,2025-07-24-16-34-01 +0000,nypost,"Family of Eliotte Heinz, student found dead in...",https://nypost.com/2025/07/24/us-news/eliotte-...
48,36,2025-07-24-15-53-50 +0000,nypost,Curtis Sliwa finally gets sit-down with NYC bu...,https://nypost.com/2025/07/24/business/curtis-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
